# NIST Uncertainty Example
This notebook is a set of examples for adding SNIST and Type Uncertainties to Files and Plotting them
Written by AWS 11/2016


In [1]:
# import 
import re
import sys
import os
from pyMeasure import *
from pyMeasure.Code.Analysis.NISTUncertainty import *
import numpy as np

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [2]:
# import a raw data file 
raw_one_port=OnePortRawModel(os.path.join(TESTS_DIRECTORY,'OnePortRawTestFile.txt'))
raw_one_port.show()

In [7]:
raw_one_port.metadata

{'Calibration_Date': '10 Aug 1998',
 'Calibration_Name': 'C980810.a1',
 'Connector_Type_Calibration': 'Type N Fem',
 'Connector_Type_Measurement': 'Type N',
 'Device_Description': 'n ck std open',
 'Device_Id': 'CTN112',
 'Measurement_Date': '8 Jun 2000',
 'Measurement_Time': '16:27:53',
 'Measurement_Type': '1-port',
 'Nbs': '1',
 'Number_Connects': '3',
 'Number_Frequencies': '28',
 'Number_Repeats': '1',
 'Operator': 'afm',
 'Port_Used': '1',
 'Program_Revision': '960910.143',
 'Program_Used': 'MEAS95',
 'Start_Frequency': '7',
 'System_Id': 'System I',
 'System_Letter': 'A'}

In [10]:
# now use the S_NIST function to create uncertainties
uncertainty_list=[]
uncertainty_mag=[]
uncertainty_phase=[]
for index,frequency in enumerate(raw_one_port.getColumn('Frequency')):
    row=raw_one_port.data[index]
    type_connector=raw_one_port.metadata["Connector_Type_Measurement"]
    arguments={'frequency':frequency,'parameter':'S11','magnitude':row[3],
               'wr_connector_type':type_connector}
    uncertainty=S_NIST(**arguments)
    uncertainty_mag.append(uncertainty[0])
    uncertainty_phase.append(uncertainty[1])
    uncertainty_list.append(uncertainty)


In [15]:
import matplotlib.pyplot as plt
frequency=raw_one_port.getColumn('Frequency')
direct_calc=[10.0**(-3.327+.046*f) for f in frequency]
plt.plot(frequency,uncertainty_mag)
plt.plot(frequency,direct_calc)
plt.show()


In [12]:
import matplotlib.pyplot as plt
plt.plot(raw_one_port.getColumn('Frequency'),uncertainty_phase)
plt.show()

In [21]:
# now use the S_NIST function to create uncertainties
b_uncertainty_list=[]
b_uncertainty_mag=[]
b_uncertainty_phase=[]

for index,frequency in enumerate(raw_one_port.getColumn('Frequency')):
    row=raw_one_port.data[index]
    type_connector=raw_one_port.metadata["Connector_Type_Measurement"]
    arguments={'frequency':frequency,'parameter':'S11','magnitude':row[3],
               'wr_connector_type':type_connector}
    uncertainty=type_b(**arguments)
    b_uncertainty_mag.append(uncertainty[0])
    b_uncertainty_phase.append(uncertainty[1])
    b_uncertainty_list.append(uncertainty)
frequency=raw_one_port.getColumn('Frequency')
plt.plot(frequency,b_uncertainty_mag)
plt.show()

In [20]:
len(frequency)

TypeError: object of type 'float' has no len()

In [23]:
print raw_one_port

#System I
#A
#Type N Fem
#Type N
#1-port
#8 Jun 2000
#16:27:53
#MEAS95
#960910.143
#afm
#C980810.a1
#10 Aug 1998
#1
#3
#1
#1
#28
#7
#n ck std open
#CTN112
Frequency,Direction,Connect,mag,arg
0.01000,1,1,0.9996,-0.47
0.01000,1,2,0.9996,-0.48
0.01000,1,3,0.9996,-0.48
0.02000,1,1,1.0001,-0.91
0.02000,1,2,1.0001,-0.92
0.02000,1,3,1.0001,-0.92
0.03000,1,1,0.9998,-1.35
0.03000,1,2,0.9998,-1.35
0.03000,1,3,0.9998,-1.35
0.04000,1,1,0.9997,-1.80
0.04000,1,2,0.9997,-1.80
0.04000,1,3,0.9997,-1.80
0.05000,1,1,0.9998,-2.25
0.05000,1,2,0.9998,-2.25
0.05000,1,3,0.9998,-2.25
0.06000,1,1,0.9999,-2.70
0.06000,1,2,0.9999,-2.70
0.06000,1,3,0.9999,-2.70
0.07000,1,1,1.0000,-3.15
0.07000,1,2,0.9999,-3.15
0.07000,1,3,0.9999,-3.15
0.08000,1,1,1.0000,-3.60
0.08000,1,2,1.0000,-3.60
0.08000,1,3,0.9999,-3.60
0.09000,1,1,1.0001,-4.04
0.09000,1,2,1.0000,-4.04
0.09000,1,3,1.0000,-4.04
0.10000,1,1,0.9998,-4.49


In [3]:
frequency=.08000
filter(lambda x:x[0]==frequency,raw_one_port.data)


[[0.08, 1, 1, 1.0, -3.6], [0.08, 1, 2, 1.0, -3.6], [0.08, 1, 3, 0.9999, -3.6]]

In [84]:
def frequency_model_collapse_multiple_measurements(model,**options):
    """Returns a model with a single set of frequencies. Default is to average values together
    but geometric mean and median are options. Geometric means of odd number of negative values fails"""
    defaults={"method":"mean"}
    # load other options from model
    for option,value in model.options.iteritems():
        if not re.search('begin_line|end_line',option):
            defaults[option]=value
    defaults["header"]=model.header
    defaults["column_names"]=model.column_names
    defaults["footer"]=model.footer
    defaults["metadata"]=model.metadata

    collapse_options={}
    for key,value in defaults.iteritems():
        collapse_options[key]=value
    for key,value in options.iteritems():
        collapse_options[key]=value    
    unique_frequency_list=list(set(model["Frequency"]))
    frequency_selector=model.column_names.index("Frequency")
    out_data=[]
    for index, frequency in enumerate(unique_frequency_list):
        data_row=filter(lambda x: x[frequency_selector]==frequency,model.data)
        if re.search('mean|av',collapse_options["method"],re.IGNORECASE):
            new_row=np.mean(np.array(data_row),axis=0).tolist()
        elif re.search('median',collapse_options["method"],re.IGNORECASE):
            new_row=np.median(np.array(data_row),axis=0).tolist()
        elif re.search('geometric',collapse_options["method"],re.IGNORECASE):
            new_row=gmean(np.array(data_row),axis=0).tolist()
        elif re.search('st',collapse_options["method"],re.IGNORECASE):
            new_row=np.std(np.array(data_row),axis=0).tolist()
        
        out_data.append(new_row)

    collapse_options["data"]=out_data

    if collapse_options["specific_descriptor"]:
        collapse_options["specific_descriptor"]=collapse_options["method"]+"_"+\
        collapse_options["specific_descriptor"]
        
    
    resulting_model=AsciiDataTable(None,**collapse_options)
    return resulting_model
def frequency_model_difference(model_1,model_2,**options):
    """Takes the difference of two models that both have frequency and a similar set of columns. Returns an object that is 
    a list of [[frequency,column_1,..column_n],...] where columns are the same in the models. If  a particular subset of 
    columns is desired use columns=["Frequency","magS11] models can be any subclass of AsciiDataTable, SNP, or 
    pandas.DataFrame, if a column is a non-numeric type it drops it. The frequency list should be unique 
    (no multiple frequencies) for at least one model"""
    # Set up defaults and pass options
    defaults={"columns":"all","interpolate":False,"average":True}
    difference_options={}
    for key,value in defaults.iteritems():
        difference_options[key]=value
    for key,value in options.iteritems():
        difference_options[key]=value
        
    # first check type, if it is a panadas data frame a little conversion is needed, else is for all other models
    if type(model_1) in [pd.DataFrame]:
        model_1=DataFrame_to_AsciiDataTable(model_1)
    if type(model_2) in [pd.DataFrame]:
        model_2=DataFrame_to_AsciiDataTable(model_2)
    # now start with a set of frequencies (unique values from both)
    frequency_set_1=set(model_1["Frequency"])
    frequency_set_2=set(model_2["Frequency"])
    model_2_frequency_selector=model_2.column_names.index('Frequency')
    column_names_set_1=set(model_1.column_names)
    column_names_set_2=set(model_2.column_names)
    
#     if list(frequency_set_1)!=model_1["Frequency"] and list(frequency_set_2)!=model_2["Frequency"]:
#         raise TypeError("The Frequency List of one of the models must be unique ")
    # All points must be in the intersection to be used
    frequency_intersection=list(frequency_set_1.intersection(frequency_set_2))
    column_names_intersection=list(column_names_set_1.intersection(column_names_set_2))
    
    difference_data=[]
    for row_index,frequency in enumerate(model_1["Frequency"]):
        new_row=[frequency]
        new_column_names=["Frequency"]
        if frequency in frequency_intersection:
            model_2_frequency_row=filter(lambda x: x[model_2_frequency_selector]==frequency,model_2.data)[0]
            print("{0} is {1}".format("model_2_frequency_row",model_2_frequency_row))
            for column_index,column in enumerate(model_1.column_names):
                if column in column_names_intersection and column not in ["Frequency"]:
                    model_2_column_selector=model_2.column_names.index(column)
                    if re.search('int|float',
                                 model_1.options["column_types"][column_index],
                                 re.IGNORECASE) and re.search('int|float',
                                                              model_2.options["column_types"][model_2_column_selector],
                                                              re.IGNORECASE):
                        
                        new_row.append(model_1.data[row_index][column_index]-model_2_frequency_row[model_2_column_selector])
                        new_column_names.append(column)
                    elif difference_options["columns"] in ["all"]:
                        new_row.append(model_1.data[row_index])
                        new_column_names.append(column)
            difference_data.append(new_row)
    difference_options["column_names"]=new_column_names
    difference_options["data"]=difference_data      
    result=AsciiDataTable(None,**difference_options)
    return result

In [4]:
np.mean(np.array(filter(lambda x:x[0]==frequency,raw_one_port.data)),axis=0).tolist()

[0.08, 1.0, 2.0, 0.9999666666666668, -3.6]

In [50]:
collapsed_one_port=frequency_model_collapse_multiple_measurements(raw_one_port,method='average')

In [55]:
import pandas as pd

In [85]:
diff= frequency_model_difference(raw_one_port,collapsed_one_port)

model_2_frequency_row is [0.01, 1, 2, 0.9996, -0.4766666666666666]
model_2_frequency_row is [0.01, 1, 2, 0.9996, -0.4766666666666666]
model_2_frequency_row is [0.01, 1, 2, 0.9996, -0.4766666666666666]
model_2_frequency_row is [0.02, 1, 2, 1.0001, -0.9166666666666666]
model_2_frequency_row is [0.02, 1, 2, 1.0001, -0.9166666666666666]
model_2_frequency_row is [0.02, 1, 2, 1.0001, -0.9166666666666666]
model_2_frequency_row is [0.03, 1, 2, 0.9998, -1.3500000000000003]
model_2_frequency_row is [0.03, 1, 2, 0.9998, -1.3500000000000003]
model_2_frequency_row is [0.03, 1, 2, 0.9998, -1.3500000000000003]
model_2_frequency_row is [0.04, 1, 2, 0.9997000000000001, -1.8]
model_2_frequency_row is [0.04, 1, 2, 0.9997000000000001, -1.8]
model_2_frequency_row is [0.04, 1, 2, 0.9997000000000001, -1.8]
model_2_frequency_row is [0.06, 1, 2, 0.9998999999999999, -2.7000000000000006]
model_2_frequency_row is [0.06, 1, 2, 0.9998999999999999, -2.7000000000000006]
model_2_frequency_row is [0.06, 1, 2, 0.9998999

In [86]:
print diff

Frequency,Direction,Connect,mag,arg
0.01,0,-1,0.0,0.00666666666667
0.01,0,0,0.0,-0.00333333333333
0.01,0,1,0.0,-0.00333333333333
0.02,0,-1,0.0,0.00666666666667
0.02,0,0,0.0,-0.00333333333333
0.02,0,1,0.0,-0.00333333333333
0.03,0,-1,0.0,2.22044604925e-16
0.03,0,0,0.0,2.22044604925e-16
0.03,0,1,0.0,2.22044604925e-16
0.04,0,-1,-1.11022302463e-16,0.0
0.04,0,0,-1.11022302463e-16,0.0
0.04,0,1,-1.11022302463e-16,0.0
0.06,0,-1,1.11022302463e-16,4.4408920985e-16
0.06,0,0,1.11022302463e-16,4.4408920985e-16
0.06,0,1,1.11022302463e-16,4.4408920985e-16
0.07,0,-1,6.66666666667e-05,0.0
0.07,0,0,-3.33333333333e-05,0.0
0.07,0,1,-3.33333333333e-05,0.0
0.08,0,-1,3.33333333332e-05,0.0
0.08,0,0,3.33333333332e-05,0.0
0.08,0,1,-6.66666666668e-05,0.0
0.1,0,0,0.0,0.0


In [5]:
np.median(np.array(filter(lambda x:x[0]==frequency,raw_one_port.data)),axis=0).tolist()

[0.08, 1.0, 2.0, 1.0, -3.6]

In [6]:
from scipy.stats.mstats import gmean
gmean(np.array(filter(lambda x:x[0]==frequency,raw_one_port.data)),axis=0).tolist()

[0.07999999999999999, 1.0, 1.8171205928321397, 0.9999666655554938, nan]

In [30]:
np.std(np.array(filter(lambda x:x[0]==frequency,raw_one_port.data)),axis=0).tolist()

[0.0, 0.0, 0.816496580927726, 4.714045207909797e-05, 0.0]

In [33]:
unique_freq_list=sorted(list(set(raw_one_port.getColumn('Frequency'))))

In [34]:
unique_freq_list

[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

In [ ]:
def calrep_raw_data(raw_model):
    "Creates a callrep type model"
    unique_frequency_list=sorted(list(set(raw_model.getColumn('Frequency'))))
    mean_data=[]
    for frequency in unique_frequency_list:
        mean_row=np.mean(np.array(filter(lambda x:x[0]==frequency,raw_model.data)),axis=0).tolist()
        new_row=[frequency,mean_row]
        